<a href="https://colab.research.google.com/github/chrober24/A.UD_289.84_seminar/blob/main/SG2_ADA_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# StyleGAN2-ADA-PyTorch



## Setup

Check GPU (V100 best. P100 and K80 acceptible)

In [ ]:
!nvidia-smi -L

In [ ]:
#@title Connect Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ## Install repo

#@markdown The next cell will install the StyleGAN repository in Google Drive. 
#@markdown If you have already installed it it will just move into that folder. 
#@markdown If you don’t have Google Drive connected it will just install the necessary code in Colab.

import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir generated
    !mkdir datasets
    !mkdir factors
    !mkdir blended_models
    !mkdir projection
    %cd projection
    !mkdir input
    !mkdir output
    %cd ../
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
    %cd ../
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir generated
    !mkdir datasets
    !mkdir factors
    !mkdir blended_models
    %cd projection
    !mkdir input
    !mkdir output
    %cd ../
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

## Dataset Preparation

Upload a .zip of square images to the `datasets` folder. You can do this manually, or use the steps in the [Dataset_Prep notebook](https://github.com/chrober24/A.UD_289.84_seminar/blob/main/Dataset_Prep.ipynb)

## Train model

In [ ]:
#@title Training Settings
#@markdown `daataset_name`: name of zipped folder of images in the datasets folder

#@markdown `resume_from`: path to either pretrained model or most recent pkl from your results folder 
#@markdown (if you’re starting a new dataset I recommend `'ffhq1024'` or `'./pretrained/wikiart.pkl'`)

#@markdown `mirror_x`: mirror augmentation of dataset. 

#@markdown `mirror_y`: vertical mirror augmentation of dataset. uncheck for images that have a specific orientation ie. faces

#@markdown `snapshot_interval`: how often the training is saved (1 is best, but you can increase to save on Drive storage)
dataset_name = 'example' #@param {type: 'string'}
dataset_path = './datasets/' + dataset_name +'.zip'
resume_from = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00016-insects_machines-mirror-11gb-gpu-gamma50-bg-resumeffhq1024/network-snapshot-000128.pkl' #@param {type: 'string'}
aug_strength = 0.2
train_count = 0
mirror_x = True #@param {type: 'boolean'}
mirror_y = False #@param {type: 'boolean'}

#optional: you might not need to edit these
gamma_value = 50.0
augs = 'bg'
config = '11gb-gpu'
snapshot_interval = 1 #@param {type: 'slider', min: 1, max: 10}

In [ ]:
#@title Run Training

!python train.py --gpus=1 --cfg=$config --metrics=None --outdir=./results --data=$dataset_path --snap=$snapshot_interval --resume=$resume_from --augpipe=$augs --initstrength=$aug_strength --gamma=$gamma_value --mirror=$mirror_x --mirrory=False --nkimg=$train_count

## Convert Legacy Model

If you have an older version of a model (Tensorflow based StyleGAN, or Runway downloaded .pkl file) you’ll need to convert to the newest version. If you’ve trained in this notebook you do **not** need to use this cell.

`--source`: path to model that you want to convert

`--dest`: path and file name to convert to.

In [ ]:
#@title Convert 

source = 'Path_to_model.pkl' #@param {type: 'string'}
dest = 'Path_to_destination.pkl' #@param {type: 'string'}

!python legacy.py --source=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/FreaGAN.pkl --dest=/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/FreaGAN_ada.pkl

## Testing/Inference

Also known as "Inference", "Evaluation" or "Testing" the model. This is the process of usinng your trained model to generate new material, usually images or videos.

In [ ]:
#@title Choose a model to generate images from
#@markdown Every script below will reference this model. To use a different model for any generation script, change this path.
network_path = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/results/00020-insects_machines-mirror-11gb-gpu-gamma50-bg-resumecustom/network-snapshot-000096.pkl' #@param {type: 'string'}

### Generate Single Images

`output`: folder to be created for current set of generated images. located in root/generated/ 

`seeds`: This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation. (format: single number: '234' list of numbers: '1,34,296' or range: '1-100'  no spaces in lists)

`truncation`: Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. The smaller the number the more realistic your images should appear, but this will also affect diversity. Most people choose between 0.5 and 1.0, but technically it's infinite. 


In [ ]:
output = 'folder_name' #@param {type: 'string'}
outdir = './generated/' + output +'/'
seeds = '0-100' #@param {type: 'string'}
truncation = 0 #@param {type: 'slider', min:0, max:1, step:0.1}



!python generate.py --outdir=$outdir --trunc=$truncation --seeds=$seeds --network=$network_path

### Truncation Traversal

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

#### Options 
`--network`: Again, this should be the path to your .pkl file.

`--seeds`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)


In [ ]:
output_folder = 'trunc-trav-1' #@param {type:'string'} 
outdir = './generated/' + output_folder + '/'
start =-5 #@param {type: 'slider', min:-5, max:5, step:0.1}
stop = 5 #@param {type: 'slider', min:-5, max:5, step:0.1}
increment = 0.5 #@param {type: 'slider', min:0, max:1, step:0.01}
seeds = '250' #@param {type: 'string'}
!python generate.py --process="truncation" --outdir=$outdir --start=$start --stop=$stop --increment=$increment --seeds=$seeds --network=$network_path

### Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at different examples of interpolation below.

#### Options



`space`: z is more entangled (random) w is less entangled (more linear, smooth)

`frames`: How many frames you want to produce. Use this to manage the length of your video.

`seeds`: two or more seeds. (follow same format as mentioned above)

`trunc`: truncation value

#### Linear Interpolation

In [ ]:
output_folder = 'folder_name' #@param {type: 'string'}
outdir = outdir = './generated/' + output_folder + '/'
space = 'z' #@param ['z', 'w']
seeds = '1,34,2,10' #@param {type: 'string'}
truncation = 5 #@param {type: 'slider', min:0, max:5, step:0.1}

!python generate.py --outdir=$outdir --space=$space --trunc=$truncation --process="interpolation" --seeds=$seeds --network=$network_path

#### Noise Loop

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--interpolation="noiseloop"`: set this to use the noise loop funtion

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--random_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

Noise loops currently only work in z space.

In [ ]:
output_folder = 'folder_name' #@param {type: 'string'}
outdir = outdir = './generated/' + output_folder + '/'
diameter = 0 #@param {type: 'slider', min:0, max:10, step:0.1}
frames = 720 #@param {type: 'number'}
truncation = 0.2 #@param {type: 'slider', min:0, max:5, step:0.1}
random_seed = 42 #@param {type: 'number'}

!python generate.py --outdir=$outdir --trunc=$truncation --process="interpolation" --interpolation="noiseloop" --diameter=$diameter --frames=$frames --random_seed=$random_seed --network=$network_path

#### Circular Loop

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [ ]:
output_folder = 'folder_name' #@param {type: 'string'}
outdir = outdir = './generated/' + output_folder + '/'
diameter = 0 #@param {type: 'slider', min:0, max:10, step:0.1}
frames = 720 #@param {type: 'number'}
truncation = 0.2 #@param {type: 'slider', min:0, max:5, step:0.1}
random_seed = 42 #@param {type: 'number'}

!python generate.py --outdir=$outdir --trunc=$truncation --process="interpolation" --interpolation="circularloop" --diameter=$diameter --frames=$frames --random_seed=$random_seed --network=$network_path

## Projection

### Basic Projector

*   `--target`: this is a path to the image file that you want to "find" in your model. This image must be the exact same size as your model.
*   `steps`: how many iterations the projctor should run for. Lower will mean less steps and less likelihood of a good projection. Higher will take longer but will likely produce better images.
*    `seed`: random seed interger value (use only one number)
*    `save_video`: Save an mp4 video of optimization progress



In [ ]:
output_folder = 'bug' #@param {type: 'string'}
outdir = './projection/output/' + output_folder + '/'
target_image = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/projection/input/bug.png' #@param {type: 'string'}
target = './projection/input/' + target_image + '.png' 
steps = 50 #@param {type: 'number'}
seed = 0 #@param {type: 'number'}
save_video = True #@param {type: 'boolean'}

!python projector.py --network=$network_path --outdir=$outdir --target=$target_image --num-steps=$steps --seed=$seed

## Feature Extraction using Closed Form Factorization

Feature Extraction is the process of finding “human readable” vectors in a StyleGAN model. For example, let’s say you wanted to find a vector that could open or close a mouth in a face model.

The feature extractor tries to automate the procss of finding important vectors in your model.

`factor_name`: name of the feature vector file to output. (saved in ./factors)

In [ ]:
#@title Generate Feature Vector File
factor_name = 'factor_name' #@param {type: 'string'}
out = './factors/' + factor_name + '.pt'

!python closed_form_factorization.py --out=$out --ckpt=$network_path

This process just created the vctor values, but we need to test it on some seed values to determine what each vector actually changes. The `apply_factor.py` script does this.

Arguments to try:


*   `i`: This stands for index. By default, the cell above will produce 512 vectors, so `-i` can be any value from 0 to 511. I recommend starting with a higher value.
*   `d`: This stands for degrees. This means how much change you want to see along th vector. I recommend a value between 5 and 10 to start with.
*   `seeds`: You know what these are by now right? :)
*   `output`: where to save the images/video
*   `space`: By default this will use the w space to reduce entanglement
*   `factor_file`: path to .pt file created in the previous step



In [ ]:
i = 200 #@param {type: 'slider', min:0, max:511, step:1}
d = 10 #@param {type: 'slider', min:1, max:100, step:1}
seeds ='250' #@param {type: 'string'}
output_folder = 'test_factors' #@param {type: 'string'}
outdir = './generated/' + output_folder + '/'
space = 'z' #@param ['z', 'w']
factor_file = 'factor_name' #@param {type: 'string'}
feature_pt = './factors/' + factor_file + '.pt'

!python apply_factor.py -i $i -d $d --seeds $seeds --ckpt $network_path $feature_pt --output $outdir --video

That just produced images or video for a single vector, but there are 511 more! To generate every vector, you can use the cell below. Update any arguments you want, `i` is set for you to the entire 0-511 range

**Warning:** This takes a long time, especially if you have more than one seed value (pro tip: don’t usee more than one seed value)! Also, this will take up a good amount of space in Google Drive. You’ve been warned!

In [ ]:
d = 10 #@param {type: 'slider', min:1, max:100, step:1}
seeds ='250' #@param {type: 'string'}
output_folder = 'test_factors' #@param {type: 'string'}
outdir = './generated/' + output_folder + '/'
space = 'z' #@param ['z', 'w']
factor_file = 'factor_name' #@param {type: 'string'}
feature_pt = './factors/' + factor_file + '.pt'

for i in range(512):
  !python apply_factor.py -i {i} -d $d --seeds $seeds --ckpt $network_path $feature_pt --output $outdir --video

# Layer Manipulations

The following scripts allow you to modify various resolution layers of the StyleGAN model.

## Flesh Digressions

Flesh Digressions works by manipulating the vectors in the base 4x4 layer. By doing this while leaving all the other layers untouched you can create a warping and twisting version of images from your model.

In [ ]:
seed = '1' #@param {type: 'string'}
truncation = 0.8 #@param {type: 'slider', min:0, max:1, step:0.1}

!python flesh_digression.py --pkl $network_path --psi $truncation --seed $seed

## Network Blending
You can take two completely different models and combine them by splitting them at a specific resolution and combining the lower layers of one model and the higher layers of another.

(Note: this tends to work best when one of the models is transfer learned from the other)

*   `lower/higher_res_model`: paths to two differrent pkl files to combine
*   `split res`: which layer to split models at. (lower layers are more structural while higher laryers are more textural)
*   `output_name`: Name of new pkl file. (will be saved in ./pretrained/)
recommended naming convention: 

    `name-of-lower-model`_`name-of-higher-model`_`split-res`

In [ ]:
lower_res_model = 'path_to_pkl' #@param {type: 'string'}
hgiher_res_model = 'path_to_pkl' #@param {type: 'string'}
split_res = 64 #@param ['4', '16', '32', '64', '128', '256', '512']
output_name = 'lower_upper_64' #@param {type: 'string'}
output_path = './pretrained/' + output_name + '.pkl'

!python blend_models.py --lower_res_pkl $lower_res_model --split_res $split_res --higher_res_pkl $hgiher_res_model --output_path $output_path

You can now take the output .pkl file and use that with any of the generation tools above.